<a href="https://www.kaggle.com/code/madeelbadar/time-series-for-sales-data?scriptVersionId=92422211" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#import all libraries
import pandas as pd
from datetime import datetime as dt
import numpy as np
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import seaborn as sns

In [2]:
#import data as csv
categories=pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
items=pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
train=pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
shop=pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')

In [3]:
#view first few rows of data
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
#turn data column into a datetime object
train['date']= pd.to_datetime(train['date'])

In [5]:
#sort values by date
train = train.sort_values(by="date")

In [6]:
#create a new column representing rev_per_day
train['rev_per_day']=(train['item_price'])*(train['item_cnt_day'])

In [7]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,rev_per_day
109593,2013-01-01,0,46,18616,349.0,1.0,349.0
85162,2013-01-01,0,54,11854,199.0,1.0,199.0
18128,2013-01-01,0,28,4906,1799.0,1.0,1799.0
112216,2013-01-01,0,42,2931,99.0,1.0,99.0
85141,2013-01-01,0,54,11604,349.0,1.0,349.0


In [8]:
#create a new df after grouping data 
train_shops= train.groupby(['date', 'shop_id']).sum().reset_index
train_overall=train.groupby(['date']).sum().reset_index()

In [9]:
train_overall.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,rev_per_day
0,2013-01-01,0,54436,17148075,1.211779e+06,1951.0,1362942.00
1,2013-01-02,3356,94801,35875911,2.246770e+06,3817.0,2536633.00
2,2013-01-03,6982,114697,35294914,2.330874e+06,3846.0,2518542.40
3,2013-01-04,7014,76857,23471233,1.655931e+06,2574.0,1823273.00
4,2013-01-05,17020,144233,44655126,2.478968e+06,4863.0,2727292.98


In [10]:
#finding the rolling mean
train_overall['rolling_mean']=train_overall['rev_per_day'].rolling(10).mean().shift(1)

In [11]:
#plotting all data points and the rolling mean
fig = px.scatter(train_overall, x="date", y="rev_per_day", title='rev per day')
fig.add_scatter(x=train_overall['date'], y=train_overall['rolling_mean'], mode='lines')
fig.show()

In [12]:
#selecting 2 columns
ready=train_overall[['rev_per_day','date']]

In [13]:
ready.head()

,rev_per_day,date
0,1362942.00,2013-01-01
1,2536633.00,2013-01-02
2,2518542.40,2013-01-03
3,1823273.00,2013-01-04
4,2727292.98,2013-01-05


In [14]:
#creating moving time horison 10 periods back as our features 
a=[]
b=[]
c=[]
d=[]
e=[]
f=[]
g=[]
h=[]
l=[]
j=[]
k=[]
for i in range (len(ready)):
    if i>9:
        a.append(ready['rev_per_day'][i])
        b.append(ready['rev_per_day'][i-1])
        c.append(ready['rev_per_day'][i-2])
        d.append(ready['rev_per_day'][i-3])
        e.append(ready['rev_per_day'][i-4])
        f.append(ready['rev_per_day'][i-5])
        g.append(ready['rev_per_day'][i-6])
        h.append(ready['rev_per_day'][i-7])
        l.append(ready['rev_per_day'][i-8])
        j.append(ready['rev_per_day'][i-9])
        k.append(ready['rev_per_day'][i-10])


    

In [15]:
ready.head(11)

,rev_per_day,date
0,1362942.00,2013-01-01
1,2536633.00,2013-01-02
2,2518542.40,2013-01-03
3,1823273.00,2013-01-04
4,2727292.98,2013-01-05
5,3264955.00,2013-01-06
6,2236610.00,2013-01-07
7,1851524.20,2013-01-08
8,3199660.53,2013-01-09
9,2580490.60,2013-01-10


In [16]:
#adding lists to our df
previous_periods = pd.DataFrame(
    {'i': a,
     'i-1': b,
     'i-2': c,
     'i-3': d,
     'i-4': e,
     'i-5': f,
     'i-6': g,
     'i-7': h,
     'i-8': l,
     'i-9': j,
     'i-10': k




    })

In [17]:
previous_periods.head()

,i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10
0,4.023868e+06,2.580491e+06,3.199661e+06,1.851524e+06,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00,1362942.00
1,5.455816e+06,4.023868e+06,2.580491e+06,3.199661e+06,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00
2,3.065623e+06,5.455816e+06,4.023868e+06,2.580491e+06,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40
3,1.976551e+06,3.065623e+06,5.455816e+06,4.023868e+06,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00
4,2.524568e+06,1.976551e+06,3.065623e+06,5.455816e+06,4023868.00,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98


In [18]:
#merging our data frames
df=ready.merge(previous_periods, left_on='rev_per_day', right_on='i')

In [19]:
df.head()

,rev_per_day,date,i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10
0,4.023868e+06,2013-01-11,4.023868e+06,2.580491e+06,3.199661e+06,1.851524e+06,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00,1362942.00
1,5.455816e+06,2013-01-12,5.455816e+06,4.023868e+06,2.580491e+06,3.199661e+06,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00
2,3.065623e+06,2013-01-13,3.065623e+06,5.455816e+06,4.023868e+06,2.580491e+06,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40
3,1.976551e+06,2013-01-14,1.976551e+06,3.065623e+06,5.455816e+06,4.023868e+06,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00
4,2.524568e+06,2013-01-15,2.524568e+06,1.976551e+06,3.065623e+06,5.455816e+06,4023868.00,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98


In [20]:
df.describe()

,rev_per_day,i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10
count,1.024000e+03,1.024000e+03,1.024000e+03,1.024000e+03,1.024000e+03,1.024000e+03,1.024000e+03,1.024000e+03,1.024000e+03,1.024000e+03,1.024000e+03,1.024000e+03
mean,3.295727e+06,3.295727e+06,3.296698e+06,3.296695e+06,3.297095e+06,3.297256e+06,3.297890e+06,3.299235e+06,3.299040e+06,3.299658e+06,3.300117e+06,3.299198e+06
std,2.265939e+06,2.265939e+06,2.265418e+06,2.265418e+06,2.265127e+06,2.265046e+06,2.264946e+06,2.264197e+06,2.264315e+06,2.264015e+06,2.263812e+06,2.264408e+06
min,1.194405e+06,1.194405e+06,1.194405e+06,1.194405e+06,1.194405e+06,1.194405e+06,1.194405e+06,1.194405e+06,1.194405e+06,1.194405e+06,1.194405e+06,1.194405e+06
25%,2.096743e+06,2.096743e+06,2.097588e+06,2.097588e+06,2.097588e+06,2.100771e+06,2.100771e+06,2.102333e+06,2.102333e+06,2.104349e+06,2.105027e+06,2.104349e+06
50%,2.668969e+06,2.668969e+06,2.668969e+06,2.668969e+06,2.668969e+06,2.668969e+06,2.671458e+06,2.673099e+06,2.673099e+06,2.673099e+06,2.673099e+06,2.673099e+06
75%,3.718098e+06,3.718098e+06,3.718098e+06,3.718098e+06,3.718098e+06,3.718098e+06,3.718098e+06,3.718098e+06,3.718098e+06,3.718098e+06,3.718098e+06,3.718098e+06
max,3.654481e+07,3.654481e+07,3.654481e+07,3.654481e+07,3.654481e+07,3.654481e+07,3.654481e+07,3.654481e+07,3.654481e+07,3.654481e+07,3.654481e+07,3.654481e+07


In [21]:
#dropping redundant columns
df=df.drop(columns=['i'])

In [22]:
#plotting current period and periods as far as 3 periods back
fig = px.scatter(df, x="date", y="rev_per_day", title='rev per day')
fig.add_scatter(x=df['date'], y=df['i-1'], mode='lines')
fig.add_scatter(x=df['date'], y=df['i-2'], mode='lines')
fig.add_scatter(x=df['date'], y=df['i-3'], mode='lines')


fig.show()

In [23]:
df.head()

,rev_per_day,date,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10
0,4.023868e+06,2013-01-11,2.580491e+06,3.199661e+06,1.851524e+06,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00,1362942.00
1,5.455816e+06,2013-01-12,4.023868e+06,2.580491e+06,3.199661e+06,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00
2,3.065623e+06,2013-01-13,5.455816e+06,4.023868e+06,2.580491e+06,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40
3,1.976551e+06,2013-01-14,3.065623e+06,5.455816e+06,4.023868e+06,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00
4,2.524568e+06,2013-01-15,1.976551e+06,3.065623e+06,5.455816e+06,4023868.00,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98


In [24]:
#seperating date time in order to be able to feed it into model
df['year'] = df['date'].apply(lambda time: time.year)

df['month'] = df['date'].apply(lambda time: time.month)

df['day'] = df['date'].apply(lambda time: time.day)
df=df.drop(columns=['date'])

In [25]:
df.head()

,rev_per_day,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,year,month,day
0,4.023868e+06,2.580491e+06,3.199661e+06,1.851524e+06,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00,1362942.00,2013,1,11
1,5.455816e+06,4.023868e+06,2.580491e+06,3.199661e+06,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00,2013,1,12
2,3.065623e+06,5.455816e+06,4.023868e+06,2.580491e+06,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2013,1,13
3,1.976551e+06,3.065623e+06,5.455816e+06,4.023868e+06,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2013,1,14
4,2.524568e+06,1.976551e+06,3.065623e+06,5.455816e+06,4023868.00,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,2013,1,15


In [26]:
#df = pd.DataFrame(df, columns = ['rev_per_day','year','month','day','i-1','i-2','i-3','i-4','i-5','i-6','i-7','i-8','i-9','i-10'])

In [27]:
df.head()

,rev_per_day,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,year,month,day
0,4.023868e+06,2.580491e+06,3.199661e+06,1.851524e+06,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00,1362942.00,2013,1,11
1,5.455816e+06,4.023868e+06,2.580491e+06,3.199661e+06,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00,2013,1,12
2,3.065623e+06,5.455816e+06,4.023868e+06,2.580491e+06,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2013,1,13
3,1.976551e+06,3.065623e+06,5.455816e+06,4.023868e+06,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2013,1,14
4,2.524568e+06,1.976551e+06,3.065623e+06,5.455816e+06,4023868.00,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,2013,1,15


In [28]:
#creating traing and test sets and seperating features and labels
df_train=df.iloc[:900]
df_test=df.iloc[900:]
df_train_label=df_train['rev_per_day']
df_test_label=df_test['rev_per_day']
df_train=df_train.drop(columns=['rev_per_day'])
df_test=df_test.drop(columns=['rev_per_day'])

In [29]:
#listing all columns
df_train.columns

Index(['i-1', 'i-2', 'i-3', 'i-4', 'i-5', 'i-6', 'i-7', 'i-8', 'i-9', 'i-10',
       'year', 'month', 'day'],
      dtype='object')

In [30]:
df_train.head()

,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,year,month,day
0,2.580491e+06,3.199661e+06,1.851524e+06,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00,1362942.00,2013,1,11
1,4.023868e+06,2.580491e+06,3.199661e+06,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00,2013,1,12
2,5.455816e+06,4.023868e+06,2.580491e+06,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2013,1,13
3,3.065623e+06,5.455816e+06,4.023868e+06,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2013,1,14
4,1.976551e+06,3.065623e+06,5.455816e+06,4023868.00,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,2013,1,15


In [31]:
#information about dataset
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900 entries, 0 to 899
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   i-1     900 non-null    float64
 1   i-2     900 non-null    float64
 2   i-3     900 non-null    float64
 3   i-4     900 non-null    float64
 4   i-5     900 non-null    float64
 5   i-6     900 non-null    float64
 6   i-7     900 non-null    float64
 7   i-8     900 non-null    float64
 8   i-9     900 non-null    float64
 9   i-10    900 non-null    float64
 10  year    900 non-null    int64  
 11  month   900 non-null    int64  
 12  day     900 non-null    int64  
dtypes: float64(10), int64(3)
memory usage: 98.4 KB


In [32]:
#definging models
regr = RandomForestRegressor()
lasso = linear_model.Lasso()
pas=PassiveAggressiveRegressor()
xgbr = xgb.XGBRegressor() 

In [33]:
#cross validating model
random_forest_crossval=cross_val_score(regr, df_train, df_train_label,scoring='neg_mean_absolute_percentage_error', cv=10)
print(np.mean(random_forest_crossval))

-0.31598555595795713


In [34]:
lasso_crossval=cross_val_score(lasso, df_train, df_train_label,scoring='neg_mean_absolute_percentage_error', cv=10)
print(np.mean(lasso_crossval))

-0.35571077823649244


In [35]:
passiveaggresive_crossval=cross_val_score(pas, df_train, df_train_label,scoring='neg_mean_absolute_percentage_error', cv=10)
print(np.mean(passiveaggresive_crossval))

-0.4764751179043559


In [36]:
xgb_crossval=cross_val_score(xgbr, df_train, df_train_label,scoring='neg_mean_absolute_percentage_error', cv=10)
print(np.mean(xgb_crossval))

-0.29405028979850056


In [37]:
#hyperparameter optamization and cross validation
param_grid = [
{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
{'bootstrap': [True], 'n_estimators': [3, 10, 30], 'max_features': [2, 3, 4,10]},
]


grid_search = GridSearchCV(regr, param_grid, cv=10, refit=True,                          
scoring='neg_mean_absolute_percentage_error',
return_train_score=True)
grid_search.fit(df_train, df_train_label)
optimised_random_forest = grid_search.best_estimator_
print(grid_search.best_score_)

-0.3204620223037379


In [38]:
param_grid = [
{'subsample': [0.1, 0.3, 0.75], 'gamma': [0, 2, 6, 8]},
{'max_depth': [0,1, 3 ,75], 'num_parallel_tree': [1,2,10,25]},
]
grid_search = GridSearchCV(xgbr, param_grid, cv=10, refit=True,                          
scoring='neg_mean_absolute_percentage_error',
return_train_score=True)

grid_search.fit(df_train, df_train_label)
optimised_xgbr = grid_search.best_estimator_
print(grid_search.best_score_)

-0.30610142059226597


In [39]:
#creating a list that contans redicted values
prediction=optimised_xgbr.predict(df_test)

In [40]:
#merging datetime columns in order to plot
cols = ['year', 'month', 'day']
df_test['date'] = df[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
df_train['date'] = df[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)



In [41]:
#merging columns again
df_train=pd.concat([df_train, df_train_label.reindex(df_train.index)], axis=1)
df_test=pd.concat([df_test, df_test_label.reindex(df_test.index)], axis=1)

In [42]:
#converting datetime column back to object type dattime
df_train['date']= pd.to_datetime(df_train['date'])
df_test['date']= pd.to_datetime(df_test['date'])

In [43]:
df_train.head()

,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,year,month,day,date,rev_per_day
0,2.580491e+06,3.199661e+06,1.851524e+06,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00,1362942.00,2013,1,11,2013-01-11,4.023868e+06
1,4.023868e+06,2.580491e+06,3.199661e+06,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2536633.00,2013,1,12,2013-01-12,5.455816e+06
2,5.455816e+06,4.023868e+06,2.580491e+06,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2518542.40,2013,1,13,2013-01-13,3.065623e+06
3,3.065623e+06,5.455816e+06,4.023868e+06,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,1823273.00,2013,1,14,2013-01-14,1.976551e+06
4,1.976551e+06,3.065623e+06,5.455816e+06,4023868.00,2580490.60,3199660.53,1851524.20,2236610.00,3264955.00,2727292.98,2013,1,15,2013-01-15,2.524568e+06


In [44]:
#creating prediction column
df_test['prediction']=prediction

In [45]:
#score for prediction
mean_absolute_percentage_error(df_test['rev_per_day'], df_test['prediction'])

0.3793424897750924

In [46]:
#plotting train set test set and prediction

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df_train['date'], y=df_train['rev_per_day'], name="train data"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_test['date'], y=df_test['rev_per_day'], name="test data"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_test['date'], y=df_test['prediction'], name="prediction data"),
    secondary_y=False,
)
